In [62]:
%load_ext autoreload
%autoreload 2

from shrimpgrad.tensor import Tensor
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
t1 = torch.arange(10).reshape(2,5)
t2 = torch.arange(10).reshape(5,2)
t1.matmul(t2)

tensor([[ 60,  70],
        [160, 195]])

In [64]:
t1 = Tensor.arange(10).reshape(2,5)
t2 = Tensor.arange(10).reshape(5,2)
t1.matmul(t2)


tensor([[60.0, 70.0], [160.0, 195.0]])

In [68]:
t1 = torch.arange(10, dtype=torch.float32).reshape(5,2)
t2 = torch.arange(10, dtype=torch.float32).reshape(2,5)
t1.matmul(t2)

tensor([[  5.,   6.,   7.,   8.,   9.],
        [ 15.,  20.,  25.,  30.,  35.],
        [ 25.,  34.,  43.,  52.,  61.],
        [ 35.,  48.,  61.,  74.,  87.],
        [ 45.,  62.,  79.,  96., 113.]])

In [66]:
# Add @ for matmul
t1 = Tensor.arange(10).reshape(2,5)
t2 = Tensor.arange(10).reshape(5,2)

t1 @ t2


tensor([[60.0, 70.0], [160.0, 195.0]])

In [67]:
t2 @ t1

tensor([[5.0, 6.0, 7.0, 8.0, 9.0],
 [15.0, 20.0, 25.0, 30.0, 35.0],
 [25.0, 34.0, 43.0, 52.0, 61.0],
 [35.0, 48.0, 61.0, 74.0, 87.0],
 [45.0, 62.0, 79.0, 96.0, 113.0]])

In [69]:
# Dtype
t1.dtype

torch.float32

In [70]:
t1 = Tensor.arange(6, dtype=shrimp.dtypes.int32)
t1

tensor([0, 1, 2, 3, 4, 5])

In [71]:
t2 = Tensor.arange(6, dtype=shrimp.dtypes.float32)
t2

tensor([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])

In [72]:
t1 @ t2

tensor(55.0)

In [73]:
t1.reshape(3,2) @ t2.reshape(2,3)

tensor([[3.0, 4.0, 5.0],
 [9.0, 14.0, 19.0],
 [15.0, 24.0, 33.0]])

In [74]:
t1

tensor([[0, 1], [2, 3], [4, 5]])

In [75]:
t1[-1]

tensor([[4, 5]])

In [76]:
t1[-1,-1]

tensor([[5]])